<a href="https://colab.research.google.com/github/sciencebyAJ/watrs_ec_processing/blob/main/WATRS_QC_EC_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WATRS Quality Conrol and Apply PI Flag

The purpose of this script is to apply a semi-automated quality control on 1/2 hourly Ameriflux formatted data. For each EC tower we apply automated quality control procedures to flag errant measurements and manually review flagged data to either remove or apply addition quality flags. The script uses bokeh to identify start and end dates of flags and document reasons for removing data for a given tower.

This script reads in aggregated data for a given tower:
* runs 1st past quality checks including a rolling IQR filter and a function to flag spikes and values outside of realistic measurement ranges following Papale et al., 2006.
* generates interactive plots for relevant variables to guide quality PI flagging
* saves the quality controlled & PI flagged data to a csv.

The output from this step serves as an input for the fluxdata qaqc software package and weekly status report checks.

Gap-filling and integration to daily timesteps i complete in a subsequent script.



### Mount Drive and Install Packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install windrose openpyxl

In [ ]:
%%capture
!git clone https://github.com/sciencebyAJ/watrs_ec_processing.git

In [ ]:
cd /content/watrs_ec_processing/

/content/watrs_ec_processing


### Import Dependencies

In [ ]:
import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from windrose import WindroseAxes
from sklearn.linear_model import LinearRegression

import bokeh.io
import bokeh.plotting
from bokeh.models import Range1d
from bokeh.io import reset_output
from bokeh.layouts import gridplot, column
from bokeh.plotting import figure, ColumnDataSource, output_file, show, save
from bokeh.models import HoverTool, Div, Range1d
from bokeh.models.formatters import DatetimeTickFormatter

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

# Import custom functions for EC data processing
import src.combine_ec_data as watrs

### Choose Tower

In [ ]:
field = "MB_WG_ZAB" #@param ["MB_WG_ZAB", "MB_WG_SCH", "MB_LET_S1", "MB_LET_S2", "MB_CEL_S1", "MB_CEL_S2"]

In [ ]:

# Define a qc flagging sheet to use
site_meta_sheet_url = "https://docs.google.com/spreadsheets/d/1fmik1-lOcGyLyLe6RmBseVzpIddQEU-9-KNh1p_4lpU/edit?usp=sharing"
as_csv_url = site_meta_sheet_url.replace('/edit?usp=sharing','/export?format=csv&gid=0')
meta_df = pd.read_csv(as_csv_url)
# meta_df.loc[meta_df.SITE==field]
# Get Paths from meta data sheet
raw_data_path = meta_df.loc[meta_df.SITE==field]['RAW_EC_PATH'][0]
pi_flag_url = meta_df.loc[meta_df.SITE==field]['PI_FLAGGING_DOC'][0]
piflag_as_csv_url = pi_flag_url.replace('/edit?usp=sharing','/export?format=csv&gid=0')
flag_df = pd.read_csv(piflag_as_csv_url)

# current EC tower paths
in_path = raw_data_path+'/Data_processed/*_2025*.csv' # set out path for saved csv at end of code
in_fname = glob.glob(in_path)[-1] #retrieve last saved combined csv from CSUMB EC Processing Step 1
print('in file:\n\t'+in_fname)
todays_date_yyyymmdd=str(datetime.date.today().year)+str(datetime.date.today().month).zfill(2)+str(datetime.date.today().day).zfill(2)
out_fname = '/'.join(in_fname.split('/')[:-1])+'/'+field+'_pi_qc_'+todays_date_yyyymmdd+'.csv'
out_fig_path = '/'.join(in_fname.split('/')[:-4])+'/Field_QAQC_Figures/'
print('pi flagged file saved as:\n\t'+out_fname)

in file:
	/content/drive/Shareddrives/WATRS_Field_Data/Field_Data/CSUMB_WineGrape_Zabala_2023/Data_processed/MB_WG_ZAB_pi_qc_20250209.csv
pi flagged file saved as:
	/content/drive/Shareddrives/WATRS_Field_Data/Field_Data/CSUMB_WineGrape_Zabala_2023/Data_processed/MB_WG_ZAB_pi_qc_20250209.csv


### Plotting Function for QCing data

In [ ]:
def line_plot(fig, x, y, source, color, label=None, x_axis_type='date', **kwargs):
        """
        Add a single time series to a :obj:`bokeh.plotting.figure.Figure`
        object using data from a datetime indexed :obj:`pandas.DataFrame` with
        an interactive hover tool.
        Interactive hover shows the values of all time series data and date
        that is added to the figure.
        Arguments:
            fig (:obj:`bokeh.plotting.figure.Figure`): a figure instance to add
                the line to.
            x (str): name of the datetime index or column in the
                :obj:`pandas.DataFrame` containing data to plot.
            y (str): name of the column in the :obj:`pandas.DataFrame` to plot.
            source (:obj:`bokeh.models.sources.ColumnDataSource`): column data
                source created from the :obj:`pandas.DataFrame` with data to
                plot.
            color (str): color of plot line, see Bokeh for color options.
            label (str or :obj:`None`): default :obj:`None`. Label for plot
                legend (for ``y``).
            x_axis_type (:obj:`str` or :obj:`None`): default 'date'. If "date"
                then the x-axis will be formatted as month-day-year.
        Returns:
            :obj:`None`
        """
        hover_mode = kwargs.pop('mode','vline')
        if label is None:
            fig.line(x, y, source=source, color=color, **kwargs)
        else:
            #label=dict(value=label) # old label requirement bokeh < 2?
            fig.line(x,y,source=source,color=color,legend_label=label,**kwargs)
            fig.legend.location = "top_left"
            fig.legend.click_policy="hide"
        # add Hover tool with additional tips if more than one line
        if len(fig.hover) == 0:
            if x_axis_type == 'date':
                Hover = HoverTool(
                    tooltips=[
                        (x,'@{}{}'.format(x,'{%F}')),
                        (y,'@{}'.format(y))
                    ],
                    formatters={'@{}'.format(x): 'datetime'},
                    mode = hover_mode
                )
                fig.add_tools(Hover)
            else:
                Hover = HoverTool(
                    tooltips=[
                        (x,'@{}'.format(x)),
                        (y,'@{}'.format(y))
                    ],
                    mode = hover_mode
                )
                fig.add_tools(Hover)
        else:
            fig.hover[0].tooltips.append((y,'@{}'.format(y)))
        # enforce datetime x axis it date
        if x_axis_type == 'date':
            fig.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")





### Error Functions

### Site Processing

In [ ]:
df= pd.read_csv(in_fname)
df = watrs.assign_datetime_index(df)
df = watrs.convert_to_num(df)
convert_flags = {'b':'bad','g':'good'} #convert flags from b/g to bad/good
flag_df['H'] = flag_df['H'].map(convert_flags) #
flag_df['G'] = flag_df['G'].map(convert_flags)
flag_df['LE'] = flag_df['LE'].map(convert_flags)
flag_df['Netrad'] = flag_df['Netrad'].map(convert_flags)

# Predefine quality flags
df['H_pi_flag'] = 'good'
df['LE_pi_flag'] = 'good'
df['G_pi_flag'] = 'good'
df['NETRAD_pi_flag'] = 'good'

# Assign SSITC quality filter
# If needed, we can adjust threshold to 7 to be more conservative
# https://www.bayceer.uni-bayreuth.de/stoerungsoekologie/de/pub/pub/104460/Foken_etal_2012b.pdf
# Set based on Table 4.4 & 4.5 in Foken et al., 2012
SSITC_THRESH = 9

df.loc[df.H_SSITC_TEST>=SSITC_THRESH,'H_pi_flag']='bad'
df.loc[df.LE_SSITC_TEST>=SSITC_THRESH,'LE_pi_flag']='bad'

df = watrs.rolling_quantile_flagging(df,_var_='NETRAD',  verbose=True)
df = watrs.flag_spikes(df,_var_='NETRAD',z=12)

df = watrs.rolling_quantile_flagging(df,_var_='LE',verbose=True)
df = watrs.flag_spikes(df,_var_='LE')


df = watrs.rolling_quantile_flagging(df,_var_='H',verbose=True)
df = watrs.flag_spikes(df,_var_='H')

df = watrs.rolling_quantile_flagging(df,_var_='G',verbose=True)
df = watrs.flag_spikes(df,_var_='G')

#iterate over flagged dates dataframe and use date flag function
for i, day in flag_df.iterrows(): #need the i token for the index
  start_date = day['start_datetime']
  end_date = day['end_datetime']
  H_flag = day['H']
  if  H_flag == 'good':
    df = watrs.date_flag(df, 'H_pi_flag', start_date, end_date, H_flag)
  if  H_flag == 'bad':
    df = watrs.date_flag(df, 'H_pi_flag', start_date, end_date, H_flag)

for i, day in flag_df.iterrows(): #need the i token for the index
  start_date = day['start_datetime']
  end_date = day['end_datetime']
  LE_flag = day['LE']
  if LE_flag == 'good':
    df = watrs.date_flag(df, 'LE_pi_flag', start_date, end_date, LE_flag)
  if LE_flag == 'bad':
    df = watrs.date_flag(df, 'LE_pi_flag', start_date, end_date, LE_flag)

for i, day in flag_df.iterrows(): #need the i token for the index
  start_date = day['start_datetime']
  end_date = day['end_datetime']
  G_flag = day['G']
  if G_flag == 'good':
    df = watrs.date_flag(df, 'G_pi_flag', start_date, end_date, G_flag)
  if G_flag == 'bad':
    df = watrs.date_flag(df, 'G_pi_flag', start_date, end_date, G_flag)

for i, day in flag_df.iterrows(): #need the i token for the index
  start_date = day['start_datetime']
  end_date = day['end_datetime']
  Netrad_flag = day['Netrad']
  if Netrad_flag == 'good':
    df = watrs.date_flag(df, 'NETRAD_pi_flag', start_date, end_date, Netrad_flag)
  if Netrad_flag == 'bad':
    df = watrs.date_flag(df, 'NETRAD_pi_flag', start_date, end_date, Netrad_flag)



NETRAD_pi_flag already created
30D rolling window applied for IQR filtering
NETRAD_pi_flag already created
LE_pi_flag already created
30D rolling window applied for IQR filtering
LE_pi_flag already created
H_pi_flag already created
30D rolling window applied for IQR filtering
H_pi_flag already created
G_pi_flag already created
30D rolling window applied for IQR filtering
G_pi_flag already created


### PI Flags

In [ ]:
# CODE SNIPPED TO MANNUALLY FLAG TIME PERIODS
# df = watrs.date_flag(df, 'H_pi_flag', '2024-10-09', '2024-10-18', 'good')
# df = watrs.date_HH_flag(df, 'H_pi_flag', '2023-05-04'+' 11:00:00', '2023-05-04'+' 13:30:00', 'bad')

In [ ]:
# update filter plot variables to evaluate updates
df['H_filt_1']=np.array(df.H)

df.loc[df['H_pi_flag']=='bad','H_filt_1']=np.nan
df.loc[df['H_pi_flag']=='good','H_filt_1']=df.loc[df['H_pi_flag']=='good']['H']


# reassign df for plotting
Hsource = ColumnDataSource(df[df['date_time']>'2023-12-31'])


figH = figure(width=1200,
             height=300,
             x_axis_label='',
             y_axis_label='Sensible Heat Wm-2')
figH.y_range = Range1d(-100, 600)
line_plot(figH, 'date_time', 'H', Hsource, color='blue', line_width=3)
line_plot(figH, 'date_time', 'H_filt_1', Hsource, color='darkorange', line_width=2)
show(figH)

Latent Heat Evaluation

In [ ]:
# CODE SNIPPED TO MANNUALLY FLAG TIME PERIODS
# df = watrs.date_flag(df, 'LE_pi_flag', '2025-01-27', '2025-01-31', 'good')

# update filter plot variables to evaluate updates
df['LE_filt_1']=np.array(df.LE)
df.loc[df['LE_pi_flag']=='bad','LE_filt_1']=np.nan
df.loc[df['LE_pi_flag']=='good','LE_filt_1']=df.loc[df['LE_pi_flag']=='good']['LE']
# reassign df for plotting
LEsource = ColumnDataSource(df[df.date_time>='2024-01'])


#turbulent fluxes
figLE = figure(width=1200,
             height=300,
             x_axis_label='',
             y_axis_label='Latent Heat Wm-2')
figLE.y_range = Range1d(-100, 800)
line_plot(figLE, 'date_time', 'LE', LEsource, color='royalblue', line_width=2)
line_plot(figLE, 'date_time', 'LE_filt_1', LEsource, color='darkorange', line_width=2)
show(figLE)



Ground Heat Flux Evaluation

In [ ]:
# CODE SNIPPED TO MANNUALLY FLAG TIME PERIODS
df = watrs.date_flag(df, 'G_pi_flag', '2025-01-27', '2025-02-06', 'good')

# update filter plot variables to evaluate updates
df['G_filt_1']=np.array(df.G)
df.loc[df['G_pi_flag']=='bad','G_filt_1']=np.nan
df.loc[df['G_pi_flag']=='good','G_filt_1']=df.loc[df['G_pi_flag']=='good']['G']
#re-assign df for plotting
Gsource = ColumnDataSource(df[df.date_time>='2024-01'])

# surface
figG = figure(width=1200,
             height=300,
             x_axis_label='',
             y_axis_label='G Wm-2')
figG.y_range = Range1d(-200, 800)
line_plot(figG, 'date_time', 'G', Gsource, color='blue', line_width=2)
line_plot(figG, 'date_time', 'G_filt_1', Gsource, color='darkorange', line_width=2)

show(figG)

In [ ]:
# CODE SNIPPED TO MANNUALLY FLAG TIME PERIODS
# df = watrs.date_flag(df, 'NETRAD_pi_flag', '2025-01-27', '2025-02-06', 'good')

# CODE SNIPPED TO MANNUALLY FLAG TIME PERIODS
## df = date_flag(df, 'NETRAD_pi_flag', '2023-05-01', '2023-05-04', 'bad')
# for d in np.arange(1,31):
#   df = watrs.date_HH_flag(df, 'NETRAD_pi_flag', '2023-04-'+str(d).zfill(2)+' 15:30:00', '2023-04-'+str(d).zfill(2)+' 16:30:00', 'bad')
#   df = watrs.date_HH_flag(df, 'NETRAD_pi_flag', '2023-05-'+str(d).zfill(2)+' 15:30:00', '2023-05-'+str(d).zfill(2)+' 16:30:00', 'bad')
#   df = watrs.date_HH_flag(df, 'NETRAD_pi_flag', '2023-06-'+str(d).zfill(2)+' 15:30:00', '2023-06-'+str(d).zfill(2)+' 16:30:00', 'bad')
# df = watrs.date_flag(df, 'NETRAD_pi_flag', '2022-09-23', '2022-10-18', 'good')

# set filter column and flag
df['NETRAD_filt_1']=np.array(df.NETRAD)
df.loc[df['NETRAD_pi_flag']=='bad','NETRAD_filt_1']=np.nan
df.loc[df['NETRAD_pi_flag']=='good','NETRAD_filt_1']=df.loc[df['NETRAD_pi_flag']=='good']['NETRAD']
# redefine df
Rnsource = ColumnDataSource(df)#[df.date_time>='2024-01'])

# surface
figRN = figure(width=1200,
             height=300,
             x_axis_label='',
             y_axis_label='NETRAD Wm-2')
figRN.y_range = Range1d(-200, 800)
line_plot(figRN, 'date_time', 'NETRAD', Rnsource, color='blue', line_width=2)
# line_plot(figRN, 'date_time', 'NETRAD_gap_fill', source, color='green', line_width=2)
line_plot(figRN, 'date_time', 'NETRAD_filt_1', Rnsource, color='darkorange', line_width=2)

show(figRN)

Plan to add additional plots to unpack sensor anomalies:
- Temp
- RH
- CO2

### Adjust wind due to settings on Logger relation to orientation
* Need to add manual catch for Zabala wind direction change to guide wind direction filtering

In [ ]:
def get_field_name(raw_data_path):
  elements= raw_data_path.split('/')[-1].split('_')
  if len(elements)>=3:
    field_name = '_'.join(elements[:-1])
  else:
    field_name = elements[0]
  return(field_name)

field_name = get_field_name(raw_data_path)

# # Need to update wind directions based on set up error
if field_name =='CSUMB_Lettuce_Pryor':
  df.loc[df.date_time<='2023-05-03','WD']=df.loc[df.date_time<='2023-05-03'].WD-30
if field_name =='CSUMB_WineGrape_Zabala':
  df.loc[df.date_time<='2023-04-20','WD']=df.loc[df.date_time<='2023-04-20'].WD-36
df.loc[df.WD<-1,'WD']=np.nan

### Export dataframe with PI flag columns

In [ ]:
print('file saved to: \t'+out_fname)
df.to_csv(out_fname)

file saved to: 	/content/drive/Shareddrives/WATRS_Field_Data/Field_Data/CSUMB_WineGrape_Zabala_2023/Data_processed/MB_WG_ZAB_pi_qc_20250209.csv
